In [ ]:
!! pip install open-metric-learning
!! pip install pytorch-metric-learning
!! pip install pillow==9.0.0

['Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/',
 'Collecting pillow==9.0.0',
 '  Downloading Pillow-9.0.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/4.3 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.7/4.3 MB\x1b \x1b22.7 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b \x1b4.3/4.3 MB\x1b \x1b63.2 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b4.3/4.3 MB\x1b \x1b41.5 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25hInstalling collected packages: pillow',
 '  Attempting uninstall: pillow',
 '    Found existing installation: Pillow 8.4.0',
 '    Uninstalling Pillow-8.4.0:',
 '      Successfully uninstalled Pillow-8.4.0',
 'Successfully installed pillow-9.0.0']

In [ ]:
import torch
from tqdm import tqdm

from oml.datasets.base import DatasetWithLabels
from oml.losses.triplet import TripletLossWithMiner
from oml.miners.inbatch_all_tri import AllTripletsMiner
from oml.models.vit.vit import ViTExtractor
from oml.samplers.balance import BalanceSampler
from oml.datasets.base import DatasetQueryGallery
from oml.metrics.embeddings import EmbeddingMetrics
from oml.utils.dataframe_format import check_retrieval_dataframe_format
from oml.utils.misc_torch import PCA
from oml.inference.flat import inference_on_images
from oml.registry.transforms import get_transforms_for_pretrained
from oml.utils.misc_torch import pairwise_dist
from pytorch_metric_learning import losses, distances, reducers, miners
import pandas as pd
from oml.transforms.images.torchvision import get_normalisation_resize_torch
from oml.models.meta.siamese import ConcatSiamese
from oml.miners.pairs import PairsMiner
from oml.inference.flat import inference_on_dataframe
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader
from oml.retrieval.postprocessors.pairwise import PairwiseImagesPostprocessor
from pprint import pprint



In [ ]:
import pandas as pd
from pathlib import Path
import pathlib

iris_path='/content/drive/MyDrive/iris'
iris_path=pathlib.PosixPath(iris_path)
p=iris_path


root_iris_id='iris_dataset'

iris_id_csv="/content/drive/MyDrive/iris/df_iris_dataset_LR.csv"

root_iris_amf='AMF_Iris_Dataset'
iris_amf='/content/drive/MyDrive/iris/AMF_Iris_Dataset'
iris_amf_csv='/content/drive/MyDrive/iris/df_amf_LR.csv'

root_iris_mmu ='MMU-Iris-Database'
iris_mmu ='/content/drive/MyDrive/iris/MMU-Iris-Database'
iris_mmu_csv = '/content/drive/MyDrive/iris/df_mmu_LR.csv'

# open metric learning format for datasets and pipelines...
df_iris_id = pd.read_csv(iris_id_csv)
df_iris_mmu = pd.read_csv(iris_mmu_csv)
df_iris_amf = pd.read_csv(iris_amf_csv)

<H3>Формируем датафреймы с 3мя базами

In [ ]:
header="label,src_file,split,is_query,is_gallery,eye,ds,path"

<H3> Iris_id dataset

In [ ]:
#columns=['label','path','split','is_query', 'is_gallery','category','x_1','x_2','y_1','y_2']

In [ ]:
#csvpath="/content/drive/MyDrive/iris/iris_id.cs
csvpath="/content/drive/MyDrive/iris/df_iris_dataset_LR.csv"
iris_dataset_csvpath = csvpath



In [ ]:
zenica_csv='/content/drive/MyDrive/iris/zenica_iris.csv'

In [ ]:
df_zenica=pd.read_csv(zenica_csv)

In [ ]:

df = pd.read_csv(csvpath)

In [ ]:
df_iris = pd.read_csv(csvpath)

save is_query is_gallery

<h3> Теперь берем предобученную на огромном датасете (что то из торговли) модель для эмбеддингов и тренируем ее 1 эпоху (больше не надо!) на iris_id train

In [ ]:

model_types=['vits16_dino', 'vits8_dino', 'vitb16_dino', 'vitb8_dino', 'vits16_inshop', 'vits16_sop', 'vits16_cub', 'vits16_cars']
modeltype='vits16_dino'
arch='vits16'


In [ ]:
import torch
device = torch.device("cuda")

<H3> Валидность датасета проверяется тут 

In [ ]:
from pathlib import Path
from oml.utils.dataframe_format import check_retrieval_dataframe_format
p=Path(iris_path)

check_retrieval_dataframe_format(df=df_iris_id, dataset_root=p)
check_retrieval_dataframe_format(df=df_iris_amf, dataset_root=p)
check_retrieval_dataframe_format(df=df_iris_mmu, dataset_root=p)

<H3>Если из предоубученных брать веса

<H1> Из своих обученных берем веса

In [ ]:
from oml.const import CKPT_SAVE_ROOT as CKPT_DIR, MOCK_DATASET_PATH as DATA_DIR
from oml.models import ViTExtractor
from oml.registry.transforms import get_transforms_for_pretrained


# first time
model = ViTExtractor.from_pretrained("vitb8_dino")
# second time
#model = ViTExtractor(weights=CKPT_DIR / "vits16_dino.ckpt", arch="vits16", normalise_features=False)
transform, im_reader = get_transforms_for_pretrained("vitb8_dino")

https://dl.fbaipublicfiles.com/dino/dino_vitbase8_pretrain/dino_vitbase8_pretrain.pth
Checkpoint is already here.


<H3>Обучение с нуля (но предобученные веса загружаются автоматом)

Triple loss on embeddings

<H3>одна эпоха

In [ ]:
# тут НЕ сохраненные на зрачках веса грузятся
#model = ViTExtractor(modeltype, arch=arch, normalise_features=False).train()
threshold=0.4
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
#transform = get_normalisation_resize_torch(im_size=im_size)
train_dataset = DatasetWithLabels(df, dataset_root=p,transform=transform)
device = torch.device("cuda")
model.to(device)

# PML specific
distance = distances.LpDistance(p=2)
reducer = reducers.ThresholdReducer(low=0)
criterion = losses.TripletMarginLoss()

miner = miners.TripletMarginMiner(margin=threshold, distance=distance, type_of_triplets="hard")

sampler = BalanceSampler(train_dataset.get_labels(), n_labels=2, n_instances=2)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=sampler)

# Одна эпоха потом посмотрим когда метрики построим может и еще надо 
n_epoch=1

for i in range(n_epoch):
  for batch in tqdm(train_loader): 
    embeddings = model(batch["input_tensors"].to(device))
    loss = criterion(embeddings, batch["labels"], miner(embeddings, batch["labels"]))  # PML specific
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    

    

100%|██████████| 32/32 [01:10<00:00,  2.21s/it]


In [ ]:
import torch
from torchvision import models
from torchsummary import summary
p=0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        p=p+len(param.data)
print('Total:',p)        

Выходные данные были обрезаны до нескольких последних строк (5000).
         1.2496e+00,  1.2479e+00,  1.1856e+00,  1.2037e+00,  1.4792e+00,
         1.1517e+00,  1.4931e+00,  1.3981e+00,  1.1755e+00,  1.2747e+00,
         1.4512e+00,  1.6981e+00,  1.2763e+00,  1.4620e+00,  1.2311e+00,
         1.3358e+00,  1.3189e+00,  1.1688e+00,  1.1460e+00,  1.2333e+00,
         1.5392e+00,  1.3786e+00,  1.4776e+00,  1.0914e+00,  1.3205e+00,
         3.4684e-01,  1.4490e+00,  1.5615e+00,  1.7563e+00,  1.1817e+00,
         1.3835e+00,  1.2721e+00,  1.1132e+00,  1.2412e+00,  1.6685e+00,
         1.2820e+00,  1.0247e+00,  9.6705e-01,  1.3902e+00,  1.3666e+00,
         1.4107e+00,  1.3042e+00,  2.4734e+00,  1.3541e+00,  1.3032e+00,
         1.0665e+00,  1.4132e+00,  1.3919e+00,  1.2264e+00,  1.3808e+00,
         1.1046e+00,  1.2908e+00,  1.4845e+00,  1.4380e+00,  1.3227e+00,
         9.0890e-01,  1.7419e+00,  1.4287e+00,  1.2927e+00,  1.2881e+00,
         1.4099e+00,  1.3153e+00,  1.5100e+00,  1.1676e+

<H3> По слегка оттренированной
 модели смотрим какие метрики в валидации получились
 в калькуляторе

In [ ]:
miner = miners.TripletMarginMiner(margin=threshold, distance=distance, type_of_triplets="all")

In [ ]:
# создаем калькулятор метрик, в котором будем накапливать вектора

val_dataset = DatasetQueryGallery(df, dataset_root=p,transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=4)

calculator = EmbeddingMetrics(map_top_k=(5,3,2,1),cmc_top_k =(5,3,2,1),precision_top_k =(5,3,2,1) )
calculator.setup(num_samples=len(train_dataset))

with torch.no_grad():
    for batch in tqdm(val_loader):
        batch["embeddings"] = model(batch["input_tensors"].to(device))
        calculator.update_data(batch)  # накапливаем вектора



 71%|███████   | 96/135 [03:32<01:26,  2.21s/it]


RuntimeError: ignored

In [ ]:
metrics = calculator.compute_metrics()  # вычисляем метрики: cmc@k, precision@k, map@k


Metrics:
{'OVERALL': {'cmc': {1: tensor(1.),
                     2: tensor(1.),
                     3: tensor(1.),
                     5: tensor(1.)},
             'map': {1: tensor(1.),
                     2: tensor(1.),
                     3: tensor(0.9980),
                     5: tensor(0.9916)},
             'pcf': {0.5: tensor(0.0052)},
             'precision': {1: tensor(1.),
                           2: tensor(0.9902),
                           3: tensor(0.9622),
                           5: tensor(0.9629)}}}


<H3>vitb6_dino
<pre>
<code>
Metrics:
{'OVERALL': {'cmc': {1: tensor(1.),
                     2: tensor(1.),
                     3: tensor(1.),
                     5: tensor(1.)},
             'map': {1: tensor(1.),
                     2: tensor(1.),
                     3: tensor(0.9980),
                     5: tensor(0.9916)},
             'pcf': {0.5: tensor(0.0052)},
             'precision': {1: tensor(1.),
                           2: tensor(0.9902),
                           3: tensor(0.9622),
                           5: tensor(0.9629)}}}

<H3> Метрики свиду ОТЛИЧНЫЕ - спасибо за веса автору
<pre>
<code>

Metrics: vits16_dino
{'OVERALL': {'cmc': {1: tensor(0.9961),
                     2: tensor(1.),
                     3: tensor(1.),
                     5: tensor(1.)},
             'map': {1: tensor(0.9961),
                     2: tensor(0.9980),
                     3: tensor(0.9938),
                     5: tensor(0.9808)},
             'pcf': {0.5: tensor(0.0078)},
             'precision': {1: tensor(0.9961),
                           2: tensor(0.9766),
                           3: tensor(0.9362),
                           5: tensor(0.9434)}}}

Теперь вместо поиска ближайшего по расстоянию 
поищем ближайшего по методу KNN (K neaarest neibourgs) при
top_k = 3
получим всего по одной ошибке на тестовых датасетах!

<H3>Инференс по  базе MMU

In [ ]:
df=df_iris_amf

In [ ]:
# Смысл такой что по запросам query мы ищем ближайший (похоже 1) 
# но только среди элементов gallery
queries=[]
df_val=df[df['split']=='validation']
df_val["path"] = df_val["path"].apply(lambda x: p / x)
queries = df_val[df_val["is_query"]==1.0]["path"].tolist()
galleries = df_val[df_val["is_gallery"]==1.0]["path"].tolist()

args = {"num_workers": 0, "batch_size": 8}
# эмбеддинги запросов и галереи
features_queries = inference_on_images(model, paths=queries, transform=transform, **args)
features_galleries = inference_on_images(model, paths=galleries, transform=transform, **args)

use_knn = True
top_k = 3
if use_knn:
    from sklearn.neighbors import NearestNeighbors
    knn = NearestNeighbors(algorithm="auto", p=2)
    knn.fit(features_galleries)
    dists, ii_closest = knn.kneighbors(features_queries, n_neighbors=top_k, return_distance=True)

else:
    dist_mat = pairwise_dist(x1=features_queries, x2=features_galleries)
    dists, ii_closest = torch.topk(dist_mat, dim=1, k=top_k, largest=False)

print(f"Top {top_k} items closest to queries are:\n {ii_closest}")


Top 3 items closest to queries are:
 [[  2   0   1]
 [  2   1   0]
 [  4   5   3]
 [  4   5   3]
 [  7   6 140]
 [  7   6 140]
 [  7   6 139]
 [140 138 139]
 [ 11   8  10]
 [ 13  12 142]
 [ 12  83   4]
 [142 180  13]
 [ 14  15 145]
 [ 16  15  14]
 [ 17  18   3]
 [ 18  17 147]
 [ 18  17 134]
 [ 20  21  19]
 [ 20  19  21]
 [ 22  23  57]
 [ 22  23 154]
 [ 22  23 153]
 [ 24  25  26]
 [ 24  25  26]
 [ 29  30  28]
 [ 31  33  32]
 [ 31  33  32]
 [ 35  34 160]
 [ 35  34 160]
 [ 35 161 160]
 [ 36 165 163]
 [ 36 163  47]
 [ 36 163 166]
 [ 36 163  47]
 [ 38  39  37]
 [ 43  42 169]
 [ 43  42  41]
 [ 45  44 172]
 [ 45  44 170]
 [ 45  44 172]
 [ 46  47  48]
 [ 47  46  48]
 [ 50  49 176]
 [ 50  49 175]
 [ 50  49 175]
 [ 53  54  51]
 [ 55  56 181]
 [ 56  55 180]
 [ 56  55 180]
 [ 59  57  58]
 [ 59  57  58]
 [ 60 186 184]
 [ 60 187 185]
 [ 60 185 187]
 [ 60 185 187]
 [ 62  61 190]
 [ 62  61 190]
 [ 62  61 190]
 [ 65  64  63]
 [ 64  63  65]
 [ 66  67 194]
 [ 66  67 194]
 [ 66  67 194]
 [ 68  69  70]
 [ 

In [ ]:
df_queries = df_val[df_val["is_query"]==1.0]
df_galleries = df_val[df_val["is_gallery"]==1.0]

In [ ]:
ppath1=[]
ppath2=[]
ppath3=[]
predict1=[]
predict2=[]
predict3=[]
dist1=[]
dist2=[]
dist3=[]



for i3,m3 in zip(ii_closest.tolist(),dists.tolist()):
  print(i3)
  print(m3)
  predict1.append(df_galleries.iloc[i3[0]]['label'])
  predict2.append(df_galleries.iloc[i3[1]]['label'])
  predict3.append(df_galleries.iloc[i3[2]]['label'])
  ppath1.append((df_galleries.iloc[i3[0]]['src_file']))
  ppath2.append((df_galleries.iloc[i3[1]]['src_file']))
  ppath3.append((df_galleries.iloc[i3[2]]['src_file']))
  dist1.append(m3[0])
  dist2.append(m3[1])
  dist3.append(m3[2])
   
  



[2, 0, 1]
[5.937991794496997, 7.189361596554706, 7.496920907252497]
[2, 1, 0]
[6.144135769814186, 7.086531272496737, 7.337838378590285]
[4, 5, 3]
[11.538967950907494, 12.490273926100398, 12.697298522540446]
[4, 5, 3]
[7.9389776821819815, 10.02828755114731, 11.219645599915147]
[7, 6, 140]
[9.572214415607256, 15.481575629113497, 21.465822955940478]
[7, 6, 140]
[10.132230084577452, 15.530412432615833, 19.785374119604977]
[7, 6, 139]
[13.839488180762638, 15.58831092909268, 21.460486719419368]
[140, 138, 139]
[1.9073486328125e-06, 12.476787529279372, 14.18728166991974]
[11, 8, 10]
[4.186617402396337, 6.553786532983957, 6.642053212684083]
[13, 12, 142]
[16.92857886021631, 17.441868607393477, 18.39671153661241]
[12, 83, 4]
[22.330854971098496, 22.961269774730855, 23.340182071292713]
[142, 180, 13]
[19.534598931637216, 19.725666749464708, 19.818515574338875]
[14, 15, 145]
[7.633580867079882, 14.911619103147174, 15.247330228651917]
[16, 15, 14]
[9.39510336420633, 13.551429273385244, 15.42768875

In [ ]:
df_queries['predict']=predict1
df_queries['predict2']=predict2
df_queries['predict3']=predict3
df_queries['ppath']=ppath1
df_queries['ppath2']=ppath2
df_queries['ppath3']=ppath3
df_queries['dist']=dist1
df_queries['dist2']=dist2
df_queries['dist3']=dist3

<ipython-input-38-00e9fe507c25>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_queries['predict']=predict1
<ipython-input-38-00e9fe507c25>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_queries['predict2']=predict2
<ipython-input-38-00e9fe507c25>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [ ]:
df_match=df_queries[df_queries['label']==df_queries['predict']]

In [ ]:
df_match

,label,src_file,split,is_query,is_gallery,eye,ds,path,predict,predict2,predict3,ppath,ppath2,ppath3,dist,dist2,dist3
0,0,Iris_20220818_122827_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,0,0,0,Iris_20220818_122823_Left.bmp,Iris_20220818_122825_Left.bmp,Iris_20220818_122821_Left.bmp,5.937992,7.189362,7.496921
3,0,Iris_20220818_122818_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,0,0,0,Iris_20220818_122823_Left.bmp,Iris_20220818_122821_Left.bmp,Iris_20220818_122825_Left.bmp,6.144136,7.086531,7.337838
6,1,Iris_20220818_105655_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,1,1,1,Iris_20220818_105659_Left.bmp,Iris_20220818_105652_Left.bmp,Iris_20220818_105650_Left.bmp,11.538968,12.490274,12.697299
9,1,Iris_20220818_105657_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,1,1,1,Iris_20220818_105659_Left.bmp,Iris_20220818_105652_Left.bmp,Iris_20220818_105650_Left.bmp,7.938978,10.028288,11.219646
10,2,Iris_20220818_121801_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,2,2,2,Iris_20220818_121757_Left.bmp,Iris_20220818_121804_Left.bmp,Iris_20220818_121804_Right.bmp,9.572214,15.481576,21.465823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,51,Iris_20220817_123808_Right.bmp,validation,1.0,0.0,R,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/z...,51,51,51,Iris_20220817_123810_Right.bmp,Iris_20220817_123805_Right.bmp,Iris_20220817_123757_Right.bmp,8.221051,10.086164,10.464122
531,52,Iris_20220818_122133_Right.bmp,validation,1.0,0.0,R,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/z...,52,52,52,Iris_20220818_122136_Right.bmp,Iris_20220818_122132_Right.bmp,Iris_20220818_122127_Right.bmp,10.822497,12.723701,20.322333
533,52,Iris_20220818_122130_Right.bmp,validation,1.0,0.0,R,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/z...,52,52,27,Iris_20220818_122127_Right.bmp,Iris_20220818_122136_Right.bmp,Iris_20220818_121545_Right.bmp,7.961108,16.983831,20.595550
536,53,Iris_20220818_122420_Right.bmp,validation,1.0,0.0,R,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/z...,53,53,53,Iris_20220818_122416_Right.bmp,Iris_20220818_122413_Right.bmp,Iris_20220818_122405_Right.bmp,4.910732,6.825879,8.693168


In [ ]:
df_err=df_queries[df_queries['label']!=df_queries['predict']]

In [ ]:
df_err

,label,src_file,split,is_query,is_gallery,eye,ds,path,predict,predict2,predict3,ppath,ppath2,ppath3,dist,dist2,dist3


In [ ]:
df=df_iris_amf

In [ ]:
# Смысл такой что по запросам query мы ищем ближайший (похоже 1) 
# но только среди элементов gallery
queries=[]
df_val=df[df['split']=='validation']
df_val["path"] = df_val["path"].apply(lambda x: p / x)
queries = df_val[df_val["is_query"]==1.0]["path"].tolist()
galleries = df_val[df_val["is_gallery"]==1.0]["path"].tolist()

args = {"num_workers": 0, "batch_size": 8}
# эмбеддинги запросов и галереи
features_queries = inference_on_images(model, paths=queries, transform=transform, **args)
features_galleries = inference_on_images(model, paths=galleries, transform=transform, **args)

use_knn = True
top_k = 3
if use_knn:
    from sklearn.neighbors import NearestNeighbors
    knn = NearestNeighbors(algorithm="auto", p=2)
    knn.fit(features_galleries)
    dists, ii_closest = knn.kneighbors(features_queries, n_neighbors=top_k, return_distance=True)

else:
    dist_mat = pairwise_dist(x1=features_queries, x2=features_galleries)
    dists, ii_closest = torch.topk(dist_mat, dim=1, k=top_k, largest=False)

print(f"Top {top_k} items closest to queries are:\n {ii_closest}")


In [ ]:
df_queries = df_val[df_val["is_query"]==1.0]
df_galleries = df_val[df_val["is_gallery"]==1.0]

In [ ]:
ppath1=[]
ppath2=[]
ppath3=[]
predict1=[]
predict2=[]
predict3=[]
dist1=[]
dist2=[]
dist3=[]



for i3,m3 in zip(ii_closest.tolist(),dists.tolist()):
  print(i3)
  print(m3)
  predict1.append(df_galleries.iloc[i3[0]]['label'])
  predict2.append(df_galleries.iloc[i3[1]]['label'])
  predict3.append(df_galleries.iloc[i3[2]]['label'])
  ppath1.append((df_galleries.iloc[i3[0]]['src_file']))
  ppath2.append((df_galleries.iloc[i3[1]]['src_file']))
  ppath3.append((df_galleries.iloc[i3[2]]['src_file']))
  dist1.append(m3[0])
  dist2.append(m3[1])
  dist3.append(m3[2])
   

[2, 0, 1]
[5.937991794496997, 7.189361596554706, 7.496920907252497]
[2, 1, 0]
[6.144135769814186, 7.086531272496737, 7.337838378590285]
[4, 5, 3]
[11.538967950907494, 12.490273926100398, 12.697298522540446]
[4, 5, 3]
[7.9389776821819815, 10.02828755114731, 11.219645599915147]
[7, 6, 140]
[9.572214415607256, 15.481575629113497, 21.465822955940478]
[7, 6, 140]
[10.132230084577452, 15.530412432615833, 19.785374119604977]
[7, 6, 139]
[13.839488180762638, 15.58831092909268, 21.460486719419368]
[140, 138, 139]
[1.9073486328125e-06, 12.476787529279372, 14.18728166991974]
[11, 8, 10]
[4.186617402396337, 6.553786532983957, 6.642053212684083]
[13, 12, 142]
[16.92857886021631, 17.441868607393477, 18.39671153661241]
[12, 83, 4]
[22.330854971098496, 22.961269774730855, 23.340182071292713]
[142, 180, 13]
[19.534598931637216, 19.725666749464708, 19.818515574338875]
[14, 15, 145]
[7.633580867079882, 14.911619103147174, 15.247330228651917]
[16, 15, 14]
[9.39510336420633, 13.551429273385244, 15.42768875

In [ ]:
df_queries['predict']=predict1
df_queries['predict2']=predict2
df_queries['predict3']=predict3
df_queries['ppath']=ppath1
df_queries['ppath2']=ppath2
df_queries['ppath3']=ppath3
df_queries['dist']=dist1
df_queries['dist2']=dist2
df_queries['dist3']=dist3

<ipython-input-47-00e9fe507c25>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_queries['predict']=predict1
<ipython-input-47-00e9fe507c25>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_queries['predict2']=predict2
<ipython-input-47-00e9fe507c25>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [ ]:
df_match=df_queries[df_queries['label']==df_queries['predict']]

In [ ]:
df_match

,label,src_file,split,is_query,is_gallery,eye,ds,path,predict,predict2,predict3,ppath,ppath2,ppath3,dist,dist2,dist3
0,0,Iris_20220818_122827_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,0,0,0,Iris_20220818_122823_Left.bmp,Iris_20220818_122825_Left.bmp,Iris_20220818_122821_Left.bmp,5.937992,7.189362,7.496921
3,0,Iris_20220818_122818_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,0,0,0,Iris_20220818_122823_Left.bmp,Iris_20220818_122821_Left.bmp,Iris_20220818_122825_Left.bmp,6.144136,7.086531,7.337838
6,1,Iris_20220818_105655_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,1,1,1,Iris_20220818_105659_Left.bmp,Iris_20220818_105652_Left.bmp,Iris_20220818_105650_Left.bmp,11.538968,12.490274,12.697299
9,1,Iris_20220818_105657_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,1,1,1,Iris_20220818_105659_Left.bmp,Iris_20220818_105652_Left.bmp,Iris_20220818_105650_Left.bmp,7.938978,10.028288,11.219646
10,2,Iris_20220818_121801_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,2,2,2,Iris_20220818_121757_Left.bmp,Iris_20220818_121804_Left.bmp,Iris_20220818_121804_Right.bmp,9.572214,15.481576,21.465823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,51,Iris_20220817_123808_Right.bmp,validation,1.0,0.0,R,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/z...,51,51,51,Iris_20220817_123810_Right.bmp,Iris_20220817_123805_Right.bmp,Iris_20220817_123757_Right.bmp,8.221051,10.086164,10.464122
531,52,Iris_20220818_122133_Right.bmp,validation,1.0,0.0,R,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/z...,52,52,52,Iris_20220818_122136_Right.bmp,Iris_20220818_122132_Right.bmp,Iris_20220818_122127_Right.bmp,10.822497,12.723701,20.322333
533,52,Iris_20220818_122130_Right.bmp,validation,1.0,0.0,R,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/z...,52,52,27,Iris_20220818_122127_Right.bmp,Iris_20220818_122136_Right.bmp,Iris_20220818_121545_Right.bmp,7.961108,16.983831,20.595550
536,53,Iris_20220818_122420_Right.bmp,validation,1.0,0.0,R,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/z...,53,53,53,Iris_20220818_122416_Right.bmp,Iris_20220818_122413_Right.bmp,Iris_20220818_122405_Right.bmp,4.910732,6.825879,8.693168


In [ ]:
df_err=df_queries[df_queries['label']!=df_queries['predict']]

In [ ]:
df_err

,label,src_file,split,is_query,is_gallery,eye,ds,path,predict,predict2,predict3,ppath,ppath2,ppath3,dist,dist2,dist3


In [ ]:
df=df_zenica

In [ ]:
# Смысл такой что по запросам query мы ищем ближайший (похоже 1) 
# но только среди элементов gallery
queries=[]
df_val=df[df['split']=='validation']
df_val["path"] = df_val["path"].apply(lambda x: p / x)
queries = df_val[df_val["is_query"]==1.0]["path"].tolist()
galleries = df_val[df_val["is_gallery"]==1.0]["path"].tolist()

args = {"num_workers": 0, "batch_size": 8}
# эмбеддинги запросов и галереи
features_queries = inference_on_images(model, paths=queries, transform=transform, **args)
features_galleries = inference_on_images(model, paths=galleries, transform=transform, **args)

use_knn = True
top_k = 3
if use_knn:
    from sklearn.neighbors import NearestNeighbors
    knn = NearestNeighbors(algorithm="auto", p=2)
    knn.fit(features_galleries)
    dists, ii_closest = knn.kneighbors(features_queries, n_neighbors=top_k, return_distance=True)

else:
    dist_mat = pairwise_dist(x1=features_queries, x2=features_galleries)
    dists, ii_closest = torch.topk(dist_mat, dim=1, k=top_k, largest=False)

print(f"Top {top_k} items closest to queries are:\n {ii_closest}")


KeyboardInterrupt: ignored

In [ ]:
df_queries = df_val[df_val["is_query"]==1.0]
df_galleries = df_val[df_val["is_gallery"]==1.0]

In [ ]:
ppath1=[]
ppath2=[]
ppath3=[]
predict1=[]
predict2=[]
predict3=[]
dist1=[]
dist2=[]
dist3=[]



for i3,m3 in zip(ii_closest.tolist(),dists.tolist()):
  print(i3)
  print(m3)
  predict1.append(df_galleries.iloc[i3[0]]['label'])
  predict2.append(df_galleries.iloc[i3[1]]['label'])
  predict3.append(df_galleries.iloc[i3[2]]['label'])
  ppath1.append((df_galleries.iloc[i3[0]]['src_file']))
  ppath2.append((df_galleries.iloc[i3[1]]['src_file']))
  ppath3.append((df_galleries.iloc[i3[2]]['src_file']))
  dist1.append(m3[0])
  dist2.append(m3[1])
  dist3.append(m3[2])
   

[3, 0, 2]
[19.657681411513476, 23.792746533295734, 23.901643755581958]
[1, 5, 0]
[9.629078583016662, 17.221723739355696, 21.46144443908912]
[2, 0, 3]
[10.259875494664428, 12.444500203991659, 15.179114518908044]
[2, 0, 3]
[16.70107034690956, 17.04667986540469, 18.963575282630647]
[5, 1, 4]
[12.73279143240526, 20.709225221561134, 21.435023573481395]
[4, 3, 7]
[18.952274010934595, 20.342220594632806, 20.499347211167102]
[6, 4, 2]
[8.994987787532096, 10.50469756630207, 14.688824013627514]
[4, 2, 6]
[14.559915337788015, 14.747243850017366, 15.857926446315666]
[8, 9, 10]
[9.55599055914175, 9.90941870038259, 16.133587110640974]
[9, 8, 10]
[9.637403942376555, 10.524958783066749, 15.197318713962341]
[10, 11, 9]
[10.587561496164195, 11.940320705279314, 15.054565872627522]
[10, 11, 9]
[9.781170829476917, 13.799249291387534, 14.682147150409884]
[12, 13, 8]
[19.842052092714603, 20.306125802751925, 22.854987724065563]
[12, 13, 8]
[8.864199231466806, 10.775551525613542, 16.995433418423847]
[15, 14, 8

In [ ]:
df_queries['predict']=predict1
df_queries['predict2']=predict2
df_queries['predict3']=predict3
df_queries['ppath']=ppath1
df_queries['ppath2']=ppath2
df_queries['ppath3']=ppath3
df_queries['dist']=dist1
df_queries['dist2']=dist2
df_queries['dist3']=dist3

<ipython-input-56-00e9fe507c25>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_queries['predict']=predict1
<ipython-input-56-00e9fe507c25>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_queries['predict2']=predict2
<ipython-input-56-00e9fe507c25>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [ ]:
df_match=df_queries[df_queries['label']==df_queries['predict']]

In [ ]:
df_match

,label,src_file,split,is_query,is_gallery,eye,ds,path,predict,predict2,predict3,ppath,ppath2,ppath3,dist,dist2,dist3
1,5,05_right_full_7.png,validation,1.0,0.0,R,zenica,/content/drive/MyDrive/iris/ZENICA_IRIS/1920x1...,5,5,5,05_right_full_2.png,05_right_full_8.png,05_right_full_4.png,19.657681,23.792747,23.901644
3,5,05_right_full_5.png,validation,1.0,0.0,R,zenica,/content/drive/MyDrive/iris/ZENICA_IRIS/1920x1...,5,5,5,05_right_full_6.png,05_left_full_6.png,05_right_full_8.png,9.629079,17.221724,21.461444
5,5,05_right_full_3.png,validation,1.0,0.0,R,zenica,/content/drive/MyDrive/iris/ZENICA_IRIS/1920x1...,5,5,5,05_right_full_4.png,05_right_full_8.png,05_right_full_2.png,10.259875,12.444500,15.179115
7,5,05_right_full_1.png,validation,1.0,0.0,R,zenica,/content/drive/MyDrive/iris/ZENICA_IRIS/1920x1...,5,5,5,05_right_full_4.png,05_right_full_8.png,05_right_full_2.png,16.701070,17.046680,18.963575
9,5,05_left_full_7.png,validation,1.0,0.0,L,zenica,/content/drive/MyDrive/iris/ZENICA_IRIS/1920x1...,5,5,5,05_left_full_6.png,05_right_full_6.png,05_left_full_8.png,12.732791,20.709225,21.435024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,1,01_left_full_8.png,validation,1.0,0.0,L,zenica,/content/drive/MyDrive/iris/ZENICA_IRIS/1920x1...,1,1,1,01_left_full_7.png,01_right_full_5.png,01_right_full_7.png,20.680920,21.750901,21.751993
153,1,01_right_full_2.png,validation,1.0,0.0,R,zenica,/content/drive/MyDrive/iris/ZENICA_IRIS/1920x1...,1,1,1,01_right_full_5.png,01_right_full_3.png,01_right_full_7.png,13.466186,13.822556,14.040978
155,1,01_right_full_4.png,validation,1.0,0.0,R,zenica,/content/drive/MyDrive/iris/ZENICA_IRIS/1920x1...,1,1,1,01_right_full_1.png,01_right_full_3.png,01_left_full_3.png,17.474780,17.954881,19.366830
157,1,01_right_full_6.png,validation,1.0,0.0,R,zenica,/content/drive/MyDrive/iris/ZENICA_IRIS/1920x1...,1,1,1,01_right_full_7.png,01_right_full_5.png,01_right_full_1.png,11.452262,11.973839,14.140838


In [ ]:
df_err=df_queries[df_queries['label']!=df_queries['predict']]

In [ ]:
df_err

,label,src_file,split,is_query,is_gallery,eye,ds,path,predict,predict2,predict3,ppath,ppath2,ppath3,dist,dist2,dist3


<H3>Другие модели пока не проверял кром vits_16_dino  (смотрите modeltype)

In [ ]:
print(list(ViTExtractor.pretrained_models.keys()))

['vits16_dino', 'vits8_dino', 'vitb16_dino', 'vitb8_dino', 'vits16_inshop', 'vits16_sop', 'vits16_cub', 'vits16_cars']


In [ ]:
cpu = torch.device("cpu")

In [ ]:
mcpu=model.to(cpu)

In [ ]:
modelpathcpu='/content/drive/MyDrive/iris/iris_idLR_vitb8_knn3_cpu.pt'
torch.save(mcpu.state_dict(), modelpathcpu)